In [1]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

In [14]:
STEPS_NUM = 60
GENERATIONS = 40

In [15]:
# Define two strategies that players can use in the game

# myhistory is a list of the player's choices in the past
# otherhistory is a list of the opponent's choices in the past

# The function should return 0 for defection and 1 for cooperation
def always_cooperate(myhistory, otherhistory):
    # This function cooperates if the opponent cooperated more than they defected in their history
    if otherhistory.count(0) > otherhistory.count(1):
        return 0
    # If the player has been defecting too much in their history, cooperate
    elif myhistory.count(1) > myhistory.count(0):
        return 0
    else:
        return 1  # Defect otherwise

# The function should return 0 for defection and 1 for cooperation
def random_answer(myhistory, otherhistory):
    # This function returns a random answer, but it's more likely to cooperate if the opponent cooperated more in their history
    if otherhistory.count(0) > otherhistory.count(1):
        return random.choices([0, 1], weights=[0.7, 0.3], k=1)[0]  # 70% chance to cooperate
    # If the player has been cooperating too much in their history, defect
    elif myhistory.count(0) > myhistory.count(1):
        return random.choices([0, 1], weights=[0.3, 0.7], k=1)[0]  # 70% chance to defect
    else:
        return random.randint(0, 1)  # Random choice otherwise


In [16]:
# Defines the score distribution for a round of the game based on the choices of the two players.
def rozdej_skore(tah1, tah2):
    if (tah1 == 1) and (tah2 == 1):
        return 2, 2
    if (tah1 == 1) and (tah2 == 0):
        return 0, 3
    if (tah1 == 0) and (tah2 == 1):
        return 3, 0
    if (tah1 == 0) and (tah2 == 0):
        return 1, 1

In [17]:
# Function simulates a game between two players for a given number of steps. 
# It keeps track of the history of choices and calculates the total score for each player
def play(f1, f2, stepsnum):
    skore1 = 0
    skore2 = 0
    historie1 = []
    historie2 = []
    for i in range(stepsnum):
        tah1 = f1(historie1, historie2)
        tah2 = f2(historie2, historie1)
        s1, s2 = rozdej_skore(tah1, tah2)
        skore1 += s1
        skore2 += s2
        historie1.append(tah1)
        historie2.append(tah2)
    return skore1, skore2

In [18]:
# Fitness function for the genetic algorithm
# The individual is a list of two integers, the first one is the threshold for the opponent's cooperation
# The second one is the threshold for the player's defection
# The function returns the total score of the player against all the opponents
def fitness(individual):
    def strategy(myhistory, otherhistory):
        if otherhistory.count(1) > individual[0] and myhistory.count(0) > individual[1]:
            return 1
        return 0
    total_score = 0
    # Play against all the opponents
    for opponent in ucastnici:
        # Play the game and add the score to the total
        # We are only interested in the player's score
        my_score, _ = play(strategy, opponent, STEPS_NUM)
        total_score += my_score
    return total_score,

In [19]:
# Crossover function for the genetic algorithm
# It swaps the first integer between the two individuals
# The second integer is not changed
# The function returns the two individuals after the crossover
# The probability of crossover is 0.5
def one_gene_crossover(ind1, ind2):
    if random.random() < 0.5:
        ind1[0], ind2[0] = ind2[0], ind1[0]
    return ind1, ind2


In [ ]:
# Genetic algorithm
# We are trying to find the best thresholds for the player's strategy
# The thresholds are the number of times the opponent cooperated and the number of times the player defected
# The player's strategy is to cooperate if the opponent cooperated more than the threshold
# and the player defected more than the threshold
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [21]:
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, STEPS_NUM)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)
toolbox.register("mate", one_gene_crossover)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=STEPS_NUM, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [22]:
ucastnici = [always_cooperate, random_answer]
dlzka_ucastnici = len(ucastnici)
# The score of the first strategy is returned as the first value of the tuple
skores = [0 for i in range(dlzka_ucastnici)]

In [23]:
print("=========================================")
print("Tournament")
print("Game length:", STEPS_NUM)
print("-----------------------------------------")
for i in range(dlzka_ucastnici):
    for j in range(i+1, dlzka_ucastnici):
        f1 = ucastnici[i]
        f2 = ucastnici[j]
        skore1, skore2 = play(f1, f2, STEPS_NUM)
        print(f"{f1.__name__} x {f2.__name__}  {skore1} : {skore2}")
        skores[i] += skore1
        skores[j] += skore2

Tournament
Game length: 60
-----------------------------------------
always_cooperate x random_answer  62 : 59


In [ ]:
print("=========================================")
print("Final Ranking")
print("-----------------------------------------")
# Sort the players based on their total score
# The index list contains the indexes of the players sorted by their score
# The lambda function is used to sort the indexes based on the scores
# The scores are stored in the skores list
index = sorted(range(dlzka_ucastnici), key=lambda k: skores[k])
poradi = 1
for i in index:
    f = ucastnici[i]
    print(f"{poradi}. {f.__name__} : {skores[i]}")
    poradi += 1

pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

best_gen = 0
best_ind = None
best_fit = -1

# Run the genetic algorithm for a given number of generations
# The best individual is the one with the highest fitness value
for gen in range(GENERATIONS):
    print(f"Generation {gen}:")
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=1, stats=stats, halloffame=hof, verbose=True)
    # Check if the best individual in the hall of fame is better than the previous best individual
    # If it is, update the best individual
    if hof[0].fitness.values[0] > best_fit:
        best_fit = hof[0].fitness.values[0]
        best_ind = hof[0]
        best_gen = gen

In [25]:
print("=========================================")
print("Best individual was found in generation:", best_gen)
print("Best individual:", best_ind)
print("Fitness:", best_fit)

Best individual was found in generation: 0
Best individual: [24, 35]
Fitness: 122.0
